In [28]:
import importlib
import warnings
from logging import getLogger
from types import ModuleType
from process_bigraph import ProcessTypes
from bigraph_schema.protocols import function_module
from unum import Unum
from importlib import import_module
from typing import *

from wholecell.utils import units
from ecoli.shared import ecoli_core as ec
from ecoli.shared.types.register import test_register_type
from ecoli.instances.base import vivarium_base as v

In [29]:
test_register_type(ec)

In [30]:
ec.types().get('unum')

{'_apply': 'apply_unum',
 '_check': 'check_unum',
 '_divide': 'divide_unum',
 '_serialize': 'serialize_unum',
 '_deserialize': 'deserialize_unum',
 '_type': 'unum',
 '_default': 1.0 [s/mol]}

In [4]:
from vivarium import Vivarium

v = Vivarium(core=ec, processes=ec.process_registry.registry, types=ec.types())

In [5]:
v.get_processes()

,Process
0,json-emitter
1,parquet-emitter
2,two_component_system
3,toyode
4,rna_interference
5,tf_binding
6,divide
7,chemostat
8,bulk_timeline
9,global_clock


In [8]:
dir(v)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_read_emitter_config',
 'add_emitter',
 'add_object',
 'add_process',
 'composite',
 'connect_process',
 'core',
 'diagram',
 'emitter_config',
 'find_package',
 'generate',
 'generate_composite_from_document',
 'get_dataclass',
 'get_processes',
 'get_results',
 'get_schema',
 'get_state',
 'get_timeseries',
 'get_type',
 'get_types',
 'get_value',
 'make_document',
 'plot_timeseries',
 'print_processes',
 'print_types',
 'process_interface',
 'process_schema',
 'register_processes',
 'register_types',
 'reset_emitters',
 'reset_paths',
 'run',
 'save',
 'set_value',
 'step']

In [9]:
v.process_schema('grow')

'(rate:float)'

In [10]:
from process_bigraph.processes import TOY_PROCESSES

In [11]:
g = TOY_PROCESSES.get("grow")(core=ec)

In [12]:
g.inputs(), g.outputs()

({'mass': 'float'}, {'mass': 'float'})

In [13]:
schema = {'mass': ['mass']}
v.add_process('grow', 'grow', config={'rate': 0.22}, inputs=schema, outputs=schema)

In [14]:
v

Vivarium( 
{ 'global_time': 0.0,
  'grow': { 'address': 'local:grow',
            'config': {'rate': 0.22},
            'inputs': {'mass': ['mass']},
            'instance': <process_bigraph.processes.growth_division.Grow object at 0x176f537a0>,
            'interval': 1.0,
            'outputs': {'mass': ['mass']},
            'shared': None},
  'mass': 0.0})

In [15]:
v.add_emitter()

In [16]:
v

Vivarium( 
{ 'emitter': { 'address': 'local:parquet-emitter',
               'config': { 'batch_size': 400,
                           'emit': {'global_time': 'any', 'mass': 'any'},
                           'experiment_id': None,
                           'out_dir': '/Users/alexanderpatrie/Desktop/repos/v2Ecoli/storage/data',
                           'out_uri': None},
               'inputs': {'global_time': ['global_time'], 'mass': ['mass']},
               'instance': <ecoli.emitters.parquet.ParquetEmitter object at 0x13263d280>,
               'outputs': None},
  'global_time': 0.0,
  'grow': { 'address': 'local:grow',
            'config': {'rate': 0.22},
            'inputs': {'mass': ['mass']},
            'instance': <process_bigraph.processes.growth_division.Grow object at 0x176f537a0>,
            'interval': 1.0,
            'outputs': {'mass': ['mass']},
            'shared': None},
  'mass': 0.0})

In [17]:
initial_state = {"mass": 11.11, "global_time": 22.2}
for obj, val in initial_state.items():
    v.set_value([obj], val)

In [18]:
v

Vivarium( 
{ 'emitter': { 'address': 'local:parquet-emitter',
               'config': { 'batch_size': 400,
                           'emit': {'global_time': 'any', 'mass': 'any'},
                           'experiment_id': None,
                           'out_dir': '/Users/alexanderpatrie/Desktop/repos/v2Ecoli/storage/data',
                           'out_uri': None},
               'inputs': {'global_time': ['global_time'], 'mass': ['mass']},
               'instance': <ecoli.emitters.parquet.ParquetEmitter object at 0x13263d280>,
               'outputs': None},
  'global_time': 22.2,
  'grow': { 'address': 'local:grow',
            'config': {'rate': 0.22},
            'inputs': {'mass': ['mass']},
            'instance': <process_bigraph.processes.growth_division.Grow object at 0x176f537a0>,
            'interval': 1.0,
            'outputs': {'mass': ['mass']},
            'shared': None},
  'mass': 11.11})

In [23]:
import numpy as np

duration = 1
timestep = 0.02
times = np.arange(0, duration, timestep).tolist()

In [25]:
for t in times:
    v.run(t)

In [26]:
v

Vivarium( 
{ 'emitter': { 'address': 'local:parquet-emitter',
               'config': { 'batch_size': 400,
                           'emit': {'global_time': 'any', 'mass': 'any'},
                           'experiment_id': None,
                           'out_dir': '/Users/alexanderpatrie/Desktop/repos/v2Ecoli/storage/data',
                           'out_uri': None},
               'inputs': {'global_time': ['global_time'], 'mass': ['mass']},
               'instance': <ecoli.emitters.parquet.ParquetEmitter object at 0x13263d280>,
               'outputs': None},
  'global_time': 46.699999999999996,
  'grow': { 'address': 'local:grow',
            'config': {'rate': 0.22},
            'inputs': {'mass': ['mass']},
            'instance': <process_bigraph.processes.growth_division.Grow object at 0x176f537a0>,
            'interval': 1.0,
            'outputs': {'mass': ['mass']},
            'shared': None},
  'mass': 1313.2578187216811})

In [39]:
from pyarrow import fs
from ecoli.emitters.parquet import format_out_dir
from tempfile import mkdtemp
import tempfile

temp = tempfile.NamedTemporaryFile(delete=False)
out_uri = format_out_dir(temp.name, 'test')
filesys, outdir = fs.FileSystem.from_uri(out_uri)

In [53]:
d = {'x': 11.11, 'y': 2.22}
import orjson

def write_parquet(data, outdir):
    with open(outdir, 'wb') as f:
        orjson.dump(data, f, option=orjson.OPT_SERIALIZE_NUMPY)

dir(filesys)


['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_from_uri',
 '_wrap_input_stream',
 '_wrap_output_stream',
 'copy_file',
 'create_dir',
 'delete_dir',
 'delete_dir_contents',
 'delete_file',
 'equals',
 'from_uri',
 'get_file_info',
 'move',
 'normalize_path',
 'open_append_stream',
 'open_input_file',
 'open_input_stream',
 'open_output_stream',
 'type_name']

In [45]:
with open(temp.name, 'rb') as f:
    d = f.read()

In [46]:
d

b'{"x":11.11,"y":2.22}'

In [48]:
type(temp)

tempfile._TemporaryFileWrapper

In [50]:
outdir

'/var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpeax36z6r/test.dat'

In [1]:
import pickle 

simdata_path = "/Users/alexanderpatrie/Desktop/repos/v2Ecoli/reconstruction/sim_data/kb/simData.cPickle"
with open(simdata_path, 'rb') as f:
    content = pickle.load(f)

<unknown>:1: SyntaxWarning: invalid escape sequence '\s'


ImportError: cannot import name 'REVERSE_TAG' from partially initialized module 'reconstruction.ecoli.dataclasses.process.metabolism' (most likely due to a circular import) (/Users/alexanderpatrie/Desktop/repos/v2Ecoli/reconstruction/ecoli/dataclasses/process/metabolism.py)